In [612]:
import pandas as pd
import fnmatch
import os
import boto3
import io

In [ ]:
# Load and Cleansing of Training Data

In [ ]:
# Load from S3
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3

bucket_name = 'training-minutes'

df = pd.DataFrame()
for obj in s3.Bucket(bucket_name).objects.all():
    ## List all files and convert them to Dataframe
    data = s3.Bucket(bucket_name).Object(obj.key).get()
    file = pd.read_excel(io.BytesIO(data['Body'].read()), sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)

    ## Cleansing of File
    ### Remove unnecessary rows, label duration and count measure columns correctly and transpose into long
    file = file.iloc[2:]
    file = file.rename(columns=lambda c: c.replace('.1', '.count') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
    file = file.melt(file.columns[0:3])

    ### Properly Name columns and categories
    file[['skills', 'entity']] = file.variable.apply(lambda x: pd.Series(str(x).split('.')))
    file = file.drop(columns=['Category Name.duration', 'variable'])
    file = file.rename(columns={'Unnamed: 0.duration': 'team', 'Unnamed: 1.duration': 'keeper'})

    ### Pivot file for all categories and create two columns for duration in minutes and number of trainings
    file = file.pivot(index=['team', 'keeper', 'skills'], columns='entity', values='value').reset_index()

    ### Clean catgegory and date columns
    file[['team', 'club']] = file.team.str.split(' BSC', 1, expand=True)
    file[['team', 'club']] = file.team.str.split(' YB', 1, expand=True)
    file['filename'] = filename
    file[['file', 'type']] = file.filename.apply(lambda x: pd.Series(str(x).split('.')))
    file['yearmonth'] = file.file.str[-6:]
    file['date'] = pd.to_datetime(file.yearmonth, format='%Y%m', errors='coerce').dropna()
    file['year'] = pd.DatetimeIndex(file['date']).year
    file['month'] = file.date.dt.month_name()

    ### Drop all helping columns and values with NaN
    file = file.drop(columns=['filename', 'file', 'type', 'club'])
    file = file.dropna(subset=['count', 'duration'])

    ### append cleansed file into one Dataframe
    df = df.append(file)

## Save to csv file
df.to_csv('trainings.csv')
   

In [586]:
# Outdated!!! Read from local files, however files are stored on S3 from March 2021

path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data'

data = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(path) for f in fnmatch.filter(files, '*.xlsx')]

df = pd.DataFrame()
for filename in data:
    source = pd.read_excel(filename, sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)
    source = source.iloc[2:]
    source = source.rename(columns=lambda c: c.replace('.1', '.count') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
    source = source.melt(source.columns[0:3])
    source[['skills', 'entity']] = source.variable.apply(lambda x: pd.Series(str(x).split('.')))
    source = source.drop(columns=['Category Name.duration', 'variable'])
    source = source.rename(columns={'Unnamed: 0.duration': 'team', 'Unnamed: 1.duration': 'keeper'})
    source = source.pivot(index=['team', 'keeper', 'skills'], columns='entity', values='value').reset_index()
    source[['team', 'club']] = source.team.str.split(' BSC', 1, expand=True)
    source[['team', 'club']] = source.team.str.split(' YB', 1, expand=True)
    source['filename'] = filename
    source[['file', 'type']] = source.filename.apply(lambda x: pd.Series(str(x).split('.')))
    source['yearmonth'] = source.file.str[-6:]
    source['date'] = pd.to_datetime(source.yearmonth, format='%Y%m', errors='coerce').dropna()
    source['year'] = pd.DatetimeIndex(source['date']).year
    source['month'] = source.date.dt.month_name()
    source = source.drop(columns=['filename', 'file', 'type', 'club'])
    source = source.dropna(subset=['count', 'duration'])
    df = df.append(source)

#df.to_csv('trainings.csv')
